# RAG over MiniKGraph no Neo4j 

This Jupyter Notebook use the Neo4j Graph integration to interact with the Neo4j database and the CypherQAChain to query the database. Using the CypherQAChain, we can ask questions about the PetroKGraph database and get answers in natural language. 

The PetroKGraph is a knowledge graph that contains information about the oil and gas industry in Brazil. It was created using the Neo4j database and the Cypher query language. The graph contains nodes representing entities such as wells, fields, companies, and reservoirs, and relationships between these entities.

In this notebook, we will use the CypherQAChain to query the PetroKGraph database and get answers to questions about the oil and gas industry in Brazil. We will ask questions about the production of oil and gas in Brazil, the fields and reservoirs where oil and gas are extracted, and the geological characteristics of the reservoirs.

### Setup

To begin, we need to import the necessary modules and set up the environment variables.

#### Install LangChain and Neo4j

```bash
!pip install openai
!pip install langchain
!pip install neo4j
!pip install transformers
!pip install sentencepiece
pip install langchain-huggingface
!pip install langchainhub
!pip install sentence_transformers
!pip install networkx node2vec rdflib
```

In [1]:
import os
import torch
import sentence_transformers
import networkx as nx

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain.chat_models.openai import ChatOpenAI

from langchain.prompts.prompt import PromptTemplate

from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


/usr/local/lib/python3.9/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
from langchain_openai import AzureChatOpenAI
from httpx import Client
from configparser import ConfigParser, ExtendedInterpolation

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config-v1.x.ini', 'UTF-8')

CERTIFICATE_PATH = 'petrobras-ca-root.pem'

OPENAI_API_VERSION = '2024-03-01-preview'
OPENAI_BASE_URL = 'https://apit.petrobras.com.br/ia/openai/v1/openai-azure/openai/deployments'
EMBEDDINGS_MODEL= config['OPENAI']['EMBEDDINGS_MODEL']


# Exemplo de uso do GPT
DEPLOYMENT_NAME = 'gpt-4o-petrobras'
llm_petrobras = AzureChatOpenAI(
    model=DEPLOYMENT_NAME,
    openai_api_version=OPENAI_API_VERSION,
    openai_api_key=config['OPENAI']['OPENAI_API_KEY'],
    base_url=f'{OPENAI_BASE_URL}/{DEPLOYMENT_NAME}',
    http_client=Client(verify=CERTIFICATE_PATH)
)

EMBEDDINGS_DEPLOYMENT_NAME='text-embedding-petrobras'
embeddings = AzureOpenAIEmbeddings(
    model=EMBEDDINGS_DEPLOYMENT_NAME,
    openai_api_version=OPENAI_API_VERSION,
    openai_api_key=config['OPENAI']['OPENAI_API_KEY'],
    base_url=f'{OPENAI_BASE_URL}/{EMBEDDINGS_DEPLOYMENT_NAME}',
    http_client=Client(verify=CERTIFICATE_PATH)
)

print(EMBEDDINGS_MODEL)

text-embedding-petrobras


In [3]:
NEO4J_URI = 'neo4j://atn1b03n13:7687' #nodo donde esta rodando neo4j
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'Diripar8$'

## Vector similarity search in an RAG application

In [4]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

### **Create a vector index**

In [11]:
graph.query("""
  CREATE VECTOR INDEX basins IF NOT EXISTS
  FOR (n:basin)
  ON (n.rdfs_label) 
  OPTIONS { indexConfig: {
    `vector.dimensions`: 256,
    `vector.similarity_function`: 'cosine'
  }}"""
)

[]

### **Show Vector index**

In [24]:
graph.query("""
  SHOW VECTOR INDEXES YIELD name, type, entityType, labelsOrTypes, properties
 """ 
)

[{'name': 'basins',
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['basin'],
  'properties': ['embedding']},
 {'name': 'lithostratigraphic_units',
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['lithostratigraphic_unit'],
  'properties': ['embedding']}]

In [23]:
# Para eliminar un indice
graph.query("""
  DROP INDEX fields 
 """ 
)

[]

In [29]:
#Se crea el embedding vector index para el nodo asociado 
vector_index = Neo4jVector.from_existing_graph(
    embeddings ,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name='lithostratigraphic_units',
    node_label="lithostratigraphic_unit",
    text_node_properties=['_id','_uri', 'rdfs_label'],
    embedding_node_property='embedding',
)

### **Embedding**

In [6]:
import os
from neo4j import GraphDatabase, Result

uri = NEO4J_URI
username = NEO4J_USERNAME
password = NEO4J_PASSWORD

# Crear el driver de conexión
driver = GraphDatabase.driver(uri, auth=(username, password))

driver.verify_connectivity()


In [16]:
import pandas as pd

def execute_query(driver):
    with driver.session() as session:
        result = session.run("""
            MATCH (n:lithostratigraphic_unit)
            WHERE n.rdfs_label IS NOT NULL
            RETURN n._id AS ID, n.rdfs_label AS Field
            LIMIT 10
                        
        """)
        # Convert the result to a DataFrame if needed
        data = [record.data() for record in result]
        return pd.DataFrame(data)

# Execute the query and get the result
kgraph = execute_query(driver)
print((kgraph))

             ID                  Field
0  formacao_125        Fm. Água Grande
1  formacao_343  Formação Taipus-Mirim
2  formacao_122           Fm. Rio Doce
3     grupo_073          Gr. Trombetas
4  formacao_256          Fm. Caravelas
5  formacao_038              Fm. Ererê
6    membro_027            Mb. Cazumba
7     grupo_034              Gr. Purus
8  formacao_171           Fm. Algodões
9  formacao_086           Fm. Nhamundá


In [42]:
def check_index_exists(index_name):
    try:
        with driver.session() as session:
            result = session.run("SHOW INDEXES")
            indexes = result.data()
            
            for index in indexes:
                
                if index_name in index["name"]:
                    print("Index exists")
                    return True
            return False
    except Exception as e:
        print("❌ Failed to check if index exists:", str(e))
        return False

def get_existing_vector_index(index_name):
    try:
        print(f"Retrieving existing index '{index_name}'...")
        vector_index = Neo4jVector.from_existing_index(
            embeddings,
            url=NEO4J_URI,
            username=NEO4J_USERNAME,
            password=NEO4J_PASSWORD,
            index_name=index_name,
            text_node_property='rdfs_label',
        )
        print(f"✅ Successfully retrieved the existing index '{index_name}'.")
        print(f"☝️ Index Name: {vector_index.index_name}")
        print(f"☝️ Node Label: {vector_index.node_label}")
        print(f"☝️ Embedding_node_property: {vector_index.embedding_node_property}")
        print(f"☝️ Text_node_property: {vector_index.text_node_property}")
        return vector_index
    except Exception as e:
        print("Failed to retrieve existing vector index:", str(e))
        return None
    
def create_vector_index():
    try:
        print("Creating vector index... 🕑🕝🕒")
        vector_index = Neo4jVector.from_existing_graph(
                       embeddings ,
                       url=NEO4J_URI,
                       username=NEO4J_USERNAME,
                       password=NEO4J_PASSWORD,
                       index_name= index_name,
                       node_label="field",
                       text_node_properties=['_id','_uri', 'rdfs_label'],
                       embedding_node_property='embedding' )
        print("✅ Vector index created successfully! ✅")

        return vector_index
    except Exception as e:
        print("Failed to create vector index:", str(e))
        return None

def verify_embeddings():
    try:
        with driver.session() as session:
            result = session.run("""
                MATCH (n:field)
                RETURN n._id, n.rdfs_label LIMIT 25
            """)
            nodes = result.data()
            if nodes:
                print("✅ Nodes with embeddings found:", nodes)
            else:
                print("❌ No nodes with embeddings found.")
    except Exception as e:
        print("Query failed:", str(e))

# Verificar si el índice ya existe
index_name = 'fields'

if check_index_exists(index_name):
    print(f"Index '{index_name}' already exists. Skipping index creation.")
    vector_index = get_existing_vector_index(index_name)
else:
    # Crear el índice vectorial
    vector_index = create_vector_index()

# Verificar los nodos después de la creación del índice
if vector_index:
        
    verify_embeddings()
    try:
        query = """MATCH (f:field)
        RETURN f 
        LIMIT 25
        """
        response = vector_index.similarity_search_with_score(query, top_k=2 )
        print("✅Neo4jVector vector_index results:", response)
        if response:
            print("✅Similarity search results:", response[0].page_content)
        else:
            print("❌ No results found.")
    except Exception as e:
        print("Similarity search failed:", str(e))
else:
    print("❌ ❌ Skipping similarity search due to failed index creation.")

Index exists
Index 'fields' already exists. Skipping index creation.
Retrieving existing index 'fields'...
✅ Successfully retrieved the existing index 'fields'.
☝️ Index Name: fields
☝️ Node Label: field
☝️ Embedding_node_property: embedding
☝️ Text_node_property: rdfs_label
✅ Nodes with embeddings found: [{'n._id': 'CAMP_CD_CAMPO_0721', 'n.rdfs_label': 'MORRO DO BARRO'}, {'n._id': 'CAMP_CD_CAMPO_0890', 'n.rdfs_label': 'CAMARAO NORTE'}, {'n._id': 'CAMP_CD_CAMPO_0109', 'n.rdfs_label': 'SARDINHA'}, {'n._id': 'CAMP_CD_CAMPO_0108', 'n.rdfs_label': 'ÁREA BAS-064'}, {'n._id': 'CAMP_CD_CAMPO_0633', 'n.rdfs_label': 'JAPIIM'}, {'n._id': 'CAMP_CD_CAMPO_0572', 'n.rdfs_label': 'JIRIBATUBA'}, {'n._id': 'CAMP_CD_CAMPO_0888', 'n.rdfs_label': 'MANATI'}, {'n._id': 'CAMP_CD_CAMPO_0131', 'n.rdfs_label': 'AZULÃO'}]
✅Neo4jVector vector_index results: []
❌ No results found.


In [41]:
response = vector_index.similarity_search("Em que BACIA está localizado o campo CAMP_CD_CAMPO_0633?")
print(response)

[]


In [38]:
response = vector_index.similarity_search_with_score("quais são os campos na base de dados fornecida?")
print(response)

[]


In [12]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=llm_petrobras,
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)


In [57]:
vector_qa.invoke(  "Quantos e quais são as bacias na base de dados fornecida?")

{'query': 'Quantos e quais são as bacias na base de dados fornecida?',
 'result': 'Na base de dados fornecida, há duas bacias:\n\n1. AMAZONAS\n2. CAMAMU-ALMADA'}

In [62]:
vector_qa.invoke(  "Quais sao os identificadores das bacias?")

{'query': 'Quais sao os identificadores das bacias?',
 'result': 'Os identificadores das bacias fornecidos são:\n\n1. AMAZONAS: BASE_CD_BACIA_030\n2. CAMAMU-ALMADA: BASE_CD_BACIA_266'}

# **Neo4j using Cypher**

In [14]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{name:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [15]:
#Se por acaso você não encontrar a informação necessária, responda com "Não foi possível encontrar a informação requerida no grafo dado".
QA_PROMPT = """
A sua função é atraves das informações submetidas que estejam no grafo, em formato de resposta de uma query cypher é responder de forma detalhada na forma culta da lingua portuguesa. 
Utilize apenas os tipos de relacionamentos e propriedades fornecidos no grafo. Não utilize nenhum outro tipo de relacionamento ou propriedade que não seja fornecido no grafo. 

Estruture sua resposta de forma clara e organizada, fornecendo informações relevantes sobre os relacionamentos e entidades mencionadas na pergunta.
Não inclua explicações ou desculpas em suas respostas.

Informação:
{context}

Questão: {question}

Resposta ideal:
"""

qa_prompt = PromptTemplate.from_template(QA_PROMPT)

### Neo4j integration within LangChain called GraphCypherQAChain

In [16]:
chain = GraphCypherQAChain.from_llm(graph=graph, qa_prompt = qa_prompt, llm=llm_petrobras, verbose=True, cypher_prompt=CYPHER_GENERATION_PROMPT)


## 🔍 Pregunta Teste

In [17]:
chain.invoke("Quantos poços estao localizados na bacia chamada AMAZONAS?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w:well)-[:located_in]->(b:basin {rdfs_label: "AMAZONAS"})
RETURN count(w) AS numberOfWells

Full Context:
[{'numberOfWells': 220}]

> Finished chain.


{'query': 'Quantos poços estao localizados na bacia chamada AMAZONAS?',
 'result': 'Na bacia denominada AMAZONAS, encontram-se localizados um total de 220 poços.'}

In [18]:
chain.invoke("Quantos poços estao localizados na bacia chamada AMAZONAS?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:basin {rdfs_label: "AMAZONAS"})<-[:located_in]-(w:well)
RETURN count(w) AS numberOfWells
Full Context:
[{'numberOfWells': 220}]

> Finished chain.


{'query': 'Quantos poços estao localizados na bacia chamada AMAZONAS?',
 'result': 'A bacia denominada "AMAZONAS" possui um total de 220 poços localizados em sua área.'}

In [19]:
chain.invoke("Quantos poços estao localizados na bacia chamada CAMAMU-ALMADA?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:basin {rdfs_label: "CAMAMU-ALMADA"})<-[:located_in]-(w:well)
RETURN count(w) AS numberOfWells
Full Context:
[{'numberOfWells': 118}]

> Finished chain.


{'query': 'Quantos poços estao localizados na bacia chamada CAMAMU-ALMADA?',
 'result': 'A bacia denominada CAMAMU-ALMADA possui um total de 118 poços localizados em sua área.'}

In [20]:
chain.invoke("Quantos poços estao localizados no campo SARDINHA?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(f:field {rdfs_label:"SARDINHA"})
RETURN count(w) AS numberOfWells
Full Context:
[{'numberOfWells': 12}]

> Finished chain.


{'query': 'Quantos poços estao localizados no campo SARDINHA?',
 'result': 'O campo SARDINHA possui um total de 12 poços localizados em sua área.'}

In [21]:
chain.invoke("Quantos poços estao localizados no campo JAPIIM?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w:well)-[:located_in]->(f:field {rdfs_label: "JAPIIM"})
RETURN count(w) AS numberOfWells

Full Context:
[{'numberOfWells': 2}]

> Finished chain.


{'query': 'Quantos poços estao localizados no campo JAPIIM?',
 'result': 'O campo JAPIIM possui um total de 2 poços localizados nele.'}

In [22]:
chain.invoke("Quais formações estão constituidas pelo material chamado siltito?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:lithostratigraphic_unit)-[:constituted_by]->(m:siltstone)
RETURN f.rdfs_label AS formacao
Full Context:
[{'formacao': 'Fm. Nhamundá'}, {'formacao': 'Fm. Algodões'}, {'formacao': 'Mb. Cazumba'}, {'formacao': 'Fm. Itaípe'}, {'formacao': 'Gr. Brotas'}, {'formacao': 'Gr. Camamu'}, {'formacao': 'Fm. Pitinga'}, {'formacao': 'Formação Urucutuca'}, {'formacao': 'Fm. Morro do Barro'}, {'formacao': 'Membro Tinharé'}]

> Finished chain.


{'query': 'Quais formações estão constituidas pelo material chamado siltito?',
 'result': 'As formações constituídas pelo material chamado siltito são as seguintes:\n\n1. **Formação Nhamundá (Fm. Nhamundá)**\n2. **Formação Algodões (Fm. Algodões)**\n3. **Membro Cazumba (Mb. Cazumba)**\n4. **Formação Itaípe (Fm. Itaípe)**\n5. **Grupo Brotas (Gr. Brotas)**\n6. **Grupo Camamu (Gr. Camamu)**\n7. **Formação Pitinga (Fm. Pitinga)**\n8. **Formação Urucutuca**\n9. **Formação Morro do Barro (Fm. Morro do Barro)**\n10. **Membro Tinharé**\n\nEstas formações geológicas apresentam siltito como um dos materiais constituintes, o que implica que esse tipo de rocha sedimentar fina está presente em todas as formações listadas.'}

In [23]:
chain.invoke("Quantas formações estão constituidas pelo material chamado siltito?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (lu:lithostratigraphic_unit)-[:constituted_by]->(s:siltstone)
RETURN count(lu) AS numberOfFormations

Full Context:
[{'numberOfFormations': 36}]

> Finished chain.


{'query': 'Quantas formações estão constituidas pelo material chamado siltito?',
 'result': 'O número total de formações constituídas pelo material denominado siltito é 36.'}

In [74]:
chain.invoke("Quais formações estão constituidas pelo material da terra chamado arenito?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (f:lithostratigraphic_unit)-[:constituted_by]->(:sandstone)
RETURN f.rdfs_label AS formacao

Full Context:
[{'formacao': 'Formação Aliança'}, {'formacao': 'Fm. Rio de Contas'}, {'formacao': 'Formação Manacapuru'}, {'formacao': 'Gr. Tapajós'}, {'formacao': 'Fm. Afligidos'}, {'formacao': 'Formação Taipus-Mirim'}, {'formacao': 'Gr. Espírito Santo'}, {'formacao': 'Fm. Ererê'}, {'formacao': 'Fm. Caravelas'}, {'formacao': 'Fm. Itaípe'}]

> Finished chain.


{'query': 'Quais formações estão constituidas pelo material da terra chamado arenito?',
 'result': 'As formações constituídas pelo material da terra chamado arenito são as seguintes:\n\n1. **Formação Aliança**\n2. **Fm. Rio de Contas**\n3. **Formação Manacapuru**\n4. **Gr. Tapajós**\n5. **Fm. Afligidos**\n6. **Formação Taipus-Mirim**\n7. **Gr. Espírito Santo**\n8. **Fm. Ererê**\n9. **Fm. Caravelas**\n10. **Fm. Itaípe**\n\nEssas formações geológicas são compostas predominantemente de arenito, caracterizando-se por suas propriedades sedimentares e sua relevância no estudo da geologia e geomorfologia.'}

In [24]:
chain.invoke("Quais são os CAMPOS localizados na bacia AMAZONAS?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:basin {rdfs_label: "AMAZONAS"})<-[:located_in]-(f:field)
RETURN f.rdfs_label AS campo

Full Context:
[{'campo': 'JAPIIM'}, {'campo': 'AZULÃO'}]

> Finished chain.


{'query': 'Quais são os CAMPOS localizados na bacia AMAZONAS?',
 'result': 'Os campos localizados na bacia Amazonas são:\n\n1. Campo Japiim\n2. Campo Azulão'}

In [25]:
chain.invoke("Quais são os CAMPOS localizados na bacia CAMAMU-ALMADA?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:basin {rdfs_label: "CAMAMU-ALMADA"})<-[:located_in]-(f:field)
RETURN f
Full Context:
[{'f': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0721', 'rdfs_label': 'MORRO DO BARRO', '_id': 'CAMP_CD_CAMPO_0721', 'embedding': [0.022277986630797386, 0.016327109187841415, -0.028187714517116547, -0.028680983930826187, 0.01824064739048481, -0.05774376168847084, -0.05173172429203987, 0.09001590311527252, 0.021866735070943832, -0.03550747409462929, 0.034170642495155334, 0.0032517146319150925, -0.056892041116952896, 0.0007174767670221627, -0.02133934572339058, -0.017919987440109253, -0.012022317387163639, 0.01830979436635971, 0.018810756504535675, 0.0013078884221613407, 0.06987334042787552, -0.015786271542310715, -0.01220856886357069, -0.048592709004879, -0.03587553650140762, -0.012725374661386013, -0.028263945132493973, -0.03850250691175461, -0.010086935013532639, -0.0535809285938

{'query': 'Quais são os CAMPOS localizados na bacia CAMAMU-ALMADA?',
 'result': 'Os campos localizados na bacia CAMAMU-ALMADA são:\n\n1. **MORRO DO BARRO**\n   - **ID:** CAMP_CD_CAMPO_0721\n   - **URI:** [Link](http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0721)\n\n2. **CAMARÃO NORTE**\n   - **ID:** CAMP_CD_CAMPO_0890\n   - **URI:** [Link](http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0890)\n\n3. **SARDINHA**\n   - **ID:** CAMP_CD_CAMPO_0109\n   - **URI:** [Link](http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0109)\n\n4. **ÁREA BAS-064**\n   - **ID:** CAMP_CD_CAMPO_0108\n   - **URI:** [Link](http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0108)\n\n5. **JIRIBATUBA**\n   - **ID:** CAMP_CD_CAMPO_0572\n   - **URI:** [Link](http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0572)\n\n6. **MANATI**\n   - **ID:** C

## 🔍 Preguntas para avaliar

### ✅  Teste de perguntas e respostas

In [26]:
### Teste de perguntas e respostas
chain.invoke("MATCH (n:well)-[r:located_in]->(b:basin) RETURN RETURN COUNT(r) as total_relationships")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:well)-[r:located_in]->(b:basin)
RETURN COUNT(r) as total_relationships
Full Context:
[{'total_relationships': 338}]

> Finished chain.


{'query': 'MATCH (n:well)-[r:located_in]->(b:basin) RETURN RETURN COUNT(r) as total_relationships',
 'result': 'A consulta solicitada tem como objetivo contar o número total de relacionamentos do tipo "located_in" entre os nós do tipo "well" e os nós do tipo "basin" no grafo.\n\nResposta detalhada:\n\nO grafo contém um total de 338 relacionamentos do tipo "located_in". Isso significa que existem 338 conexões no grafo onde um poço ("well") está localizado em uma bacia ("basin").'}

Identificar os poços que estão localizados na bacia especificada

In [27]:
chain.invoke("Quais são os identificadores dos poços localizados na bacia com identificador BASE_CD_BACIA_030?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:basin {_id: "BASE_CD_BACIA_030"})<-[:located_in]-(w:well)
RETURN w._id
Full Context:
[{'w._id': 'POCO_CD_POCO_023016'}, {'w._id': 'POCO_CD_POCO_022806'}, {'w._id': 'POCO_CD_POCO_023017'}, {'w._id': 'POCO_CD_POCO_022886'}, {'w._id': 'POCO_CD_POCO_022809'}, {'w._id': 'POCO_CD_POCO_022875'}, {'w._id': 'POCO_CD_POCO_023011'}, {'w._id': 'POCO_CD_POCO_022758'}, {'w._id': 'POCO_CD_POCO_023059'}, {'w._id': 'POCO_CD_POCO_023037'}]

> Finished chain.


{'query': 'Quais são os identificadores dos poços localizados na bacia com identificador BASE_CD_BACIA_030?',
 'result': 'Os identificadores dos poços localizados na bacia com identificador BASE_CD_BACIA_030 são:\n\n- POCO_CD_POCO_023016\n- POCO_CD_POCO_022806\n- POCO_CD_POCO_023017\n- POCO_CD_POCO_022886\n- POCO_CD_POCO_022809\n- POCO_CD_POCO_022875\n- POCO_CD_POCO_023011\n- POCO_CD_POCO_022758\n- POCO_CD_POCO_023059\n- POCO_CD_POCO_023037'}

In [64]:
chain.invoke("Quais poços sao localizados na bacia CAMAMU-ALMADA com identificador BASE_CD_BACIA_266?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:basin {_id: "BASE_CD_BACIA_266", rdfs_label: "CAMAMU-ALMADA"})<-[:located_in]-(w:well)
RETURN w
Full Context:
[{'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_005755', 'rdfs_label': '1-BAS-79', '_id': 'POCO_CD_POCO_005755', 'embedding': [0.013798758387565613, 0.0005058402311988175, -0.047536250203847885, -0.04727967828512192, 0.0007083620293997228, -0.034381747245788574, -0.013826961629092693, 0.06024038791656494, 0.03639379143714905, -0.027609027922153473, 0.019592497497797012, 0.0020387833938002586, -0.047450922429561615, -0.001828057924285531, -0.009141869843006134, 0.0045607248321175575, -0.006302203051745892, 0.0011439681984484196, 0.011428301222622395, 0.01861529052257538, 0.04100353270769119, -0.032840874046087265, -0.016532015055418015, -0.03827150911092758, -0.008243493735790253, -0.012929648160934448, -0.03543514385819435, -0.05488970875740051, 0.01483143

{'query': 'Quais poços sao localizados na bacia CAMAMU-ALMADA com identificador BASE_CD_BACIA_266?',
 'result': 'Os poços localizados na bacia CAMAMU-ALMADA, identificada pelo código BASE_CD_BACIA_266, são:\n\n1. **Poço 1-BAS-79**\n   - Identificador: POCO_CD_POCO_005755\n\n2. **Poço 1-BAS-112**\n   - Identificador: POCO_CD_POCO_005658\n\n3. **Poço 3-MB-4-BA**\n   - Identificador: POCO_CD_POCO_005617\n\n4. **Poço 1-PCZ-1-BA**\n   - Identificador: POCO_CD_POCO_005612\n\n5. **Poço 1-COST-3-BAS**\n   - Identificador: POCO_CD_POCO_006053\n\n6. **Poço 1-BAS-83**\n   - Identificador: POCO_CD_POCO_005743\n\n7. **Poço 1-BAS-72**\n   - Identificador: POCO_CD_POCO_005648\n\n8. **Poço 7-MNT-5D-BAS**\n   - Identificador: POCO_CD_POCO_006258\n\n9. **Poço 1-BAS-77**\n   - Identificador: POCO_CD_POCO_005759\n\n10. **Poço 1-BAS-64**\n    - Identificador: POCO_CD_POCO_005960'}

### ✅ **Pregunta No. 1**

In [68]:
chain.run("Quantos poços localizados na bacia BASE_CD_BACIA_030?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:basin {_id: 'BASE_CD_BACIA_030'})<-[:located_in]-(w:well)
RETURN COUNT(w) AS total_wells

Full Context:
[{'total_wells': 220}]

> Finished chain.


'A bacia identificada pelo código BASE_CD_BACIA_030 possui um total de 220 poços localizados em seu território.'

In [69]:
chain.run("Quais sao os poços localizados na bacia BASE_CD_BACIA_030?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:basin {_id: "BASE_CD_BACIA_030"})<-[:located_in]-(w:well)
RETURN w

Full Context:
[{'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_023016', 'rdfs_label': '2-JIST-1-PA', '_id': 'POCO_CD_POCO_023016', 'embedding': [0.0015631438000127673, 0.014045652002096176, -0.026118146255612373, -0.03247372806072235, 0.005580601282417774, -0.04189777746796608, -0.0176917165517807, 0.08210714161396027, 0.028373748064041138, -0.022808391600847244, 0.03322244808077812, 0.003941547591239214, -0.038707833737134933, 0.0012013016967102885, 0.004274172242730856, 0.017192991450428963, -0.012741322629153728, 0.006260435562580824, 0.015812426805496216, 0.00021560481400229037, 0.06450540572404861, -0.017464181408286095, -0.024351133033633232, -0.019821612164378166, -0.016761736944317818, -0.015171126462519169, -0.018799781799316406, -0.05658842623233795, 0.015751125290989876, -0.030769

'A bacia identificada como "BASE_CD_BACIA_030" possui os seguintes poços localizados:\n\n1. **Poço: 2-JIST-1-PA**\n   - Identificador: POCO_CD_POCO_023016\n   - Descrição: Este poço é identificado por "2-JIST-1-PA" e possui o código único POCO_CD_POCO_023016.\n\n2. **Poço: 1-AM-13-AM**\n   - Identificador: POCO_CD_POCO_022806\n   - Descrição: Este poço é identificado por "1-AM-13-AM" e possui o código único POCO_CD_POCO_022806.\n\n3. **Poço: 2-RPST-1-PA**\n   - Identificador: POCO_CD_POCO_023017\n   - Descrição: Este poço é identificado por "2-RPST-1-PA" e possui o código único POCO_CD_POCO_023017.\n\n4. **Poço: 1-TR-1-PA**\n   - Identificador: POCO_CD_POCO_022886\n   - Descrição: Este poço é identificado por "1-TR-1-PA" e possui o código único POCO_CD_POCO_022886.\n\n5. **Poço: 1-AM-6A-AM**\n   - Identificador: POCO_CD_POCO_022809\n   - Descrição: Este poço é identificado por "1-AM-6A-AM" e possui o código único POCO_CD_POCO_022809.\n\n6. **Poço: 1-AM-10-AM**\n   - Identificador: POCO

In [465]:
chain.run("Quais sao os poços localizados na bacia BASE_CD_BACIA_116?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_116"})
RETURN w
Full Context:
[{'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_009032', '_id': 'POCO_CD_POCO_009032'}}, {'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_006882', '_id': 'POCO_CD_POCO_006882'}}, {'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_007140', '_id': 'POCO_CD_POCO_007140'}}, {'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_010831', '_id': 'POCO_CD_POCO_010831'}}]

> Finished chain.


'Com base nas informações fornecidas, os poços localizados na bacia BASE_CD_BACIA_116 são:\n- POCO_CD_POCO_009032\n- POCO_CD_POCO_006882\n- POCO_CD_POCO_007140\n- POCO_CD_POCO_010831'

### ✅ **Pregunta No. 2**

In [63]:
chain.run("Quais sao os campos que estao localizados na bacia identificada BASE_CD_BACIA_030?")

/u/f5w9/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:basin {_id: 'BASE_CD_BACIA_030'})<-[:located_in]-(f:field)
RETURN f.rdfs_label

Full Context:
[{'f.rdfs_label': 'JAPIIM'}, {'f.rdfs_label': 'AZULÃO'}]

> Finished chain.


'A bacia identificada como BASE_CD_BACIA_030 possui os seguintes campos localizados em seu território:\n\n1. **Campo Japiim**\n   - Nome: JAPIIM\n\n2. **Campo Azulão**\n   - Nome: AZULÃO'

In [65]:
chain.run("Quais os campos que estao localizados na bacia BASE_CD_BACIA_266?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:basin {_id: 'BASE_CD_BACIA_266'})<-[:located_in]-(f:field)
RETURN f.rdfs_label

Full Context:
[{'f.rdfs_label': 'MORRO DO BARRO'}, {'f.rdfs_label': 'CAMARAO NORTE'}, {'f.rdfs_label': 'SARDINHA'}, {'f.rdfs_label': 'ÁREA BAS-064'}, {'f.rdfs_label': 'JIRIBATUBA'}, {'f.rdfs_label': 'MANATI'}]

> Finished chain.


'Os campos localizados na bacia BASE_CD_BACIA_266 são:\n\n1. Morro do Barro\n2. Camarão Norte\n3. Sardinha\n4. Área BAS-064\n5. Jiribatuba\n6. Manati'

In [67]:
chain.run("Quais os ID campos localizados na bacia BASE_CD_BACIA_266?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:basin {_id: 'BASE_CD_BACIA_266'})<-[:located_in]-(f:field)
RETURN f._id

Full Context:
[{'f._id': 'CAMP_CD_CAMPO_0721'}, {'f._id': 'CAMP_CD_CAMPO_0890'}, {'f._id': 'CAMP_CD_CAMPO_0109'}, {'f._id': 'CAMP_CD_CAMPO_0108'}, {'f._id': 'CAMP_CD_CAMPO_0572'}, {'f._id': 'CAMP_CD_CAMPO_0888'}]

> Finished chain.


'Os campos localizados na bacia identificada como BASE_CD_BACIA_266 são os seguintes:\n\n1. CAMP_CD_CAMPO_0721\n2. CAMP_CD_CAMPO_0890\n3. CAMP_CD_CAMPO_0109\n4. CAMP_CD_CAMPO_0108\n5. CAMP_CD_CAMPO_0572\n6. CAMP_CD_CAMPO_0888'

### ✅ **Pregunta No. 3**

In [77]:
chain.run("Quantos poços estão localizados no campo CAMP_CD_CAMPO_0888?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(f:field {rdfs_label: "CAMP_CD_CAMPO_0888"})
RETURN count(w) AS numberOfWells
Full Context:
[{'numberOfWells': 0}]

> Finished chain.


'O campo identificado pelo código CAMP_CD_CAMPO_0888 não possui nenhum poço localizado em sua área.'

In [86]:
chain.run("Quantos poços estão localizados no campo CAMP_CD_CAMPO_0890?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:field {_id: "CAMP_CD_CAMPO_0890"})<-[:located_in]-(w:well)
RETURN count(w) AS numberOfWells
Full Context:
[{'numberOfWells': 2}]

> Finished chain.


'O campo identificado pelo código CAMP_CD_CAMPO_0890 possui um total de 2 poços.'

In [80]:
chain.run("Quantos poços estão localizados no campo CAMP_CD_CAMPO_0108?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w:well)-[:located_in]->(f:field {rdfs_label: "CAMP_CD_CAMPO_0108"})
RETURN count(w) AS numberOfWells

Full Context:
[{'numberOfWells': 0}]

> Finished chain.


'Nenhum poço está localizado no campo identificado como CAMP_CD_CAMPO_0108.'

### ✅ Pregunta No. 4

In [89]:
chain.run("Quais são os poços que estão localizados no campo CAMP_CD_CAMPO_0890?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (f:field {_id: "CAMP_CD_CAMPO_0890"})<-[:located_in]-(w:well)
RETURN w.rdfs_label AS wellName

Full Context:
[{'wellName': '3-BAS-131'}, {'wellName': '3-BRSA-135-BAS'}]

> Finished chain.


'Os poços localizados no campo denominado CAMP_CD_CAMPO_0890 são:\n\n1. Poço: 3-BAS-131\n2. Poço: 3-BRSA-135-BAS'

In [92]:
chain.run("Quantos e quais são os poços que estão localizados no campo CAMP_CD_CAMPO_0890?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:field {_id: "CAMP_CD_CAMPO_0890"})<-[:located_in]-(w:well)
RETURN w.rdfs_label AS wellName, w._id AS wellId
Full Context:
[{'wellName': '3-BAS-131', 'wellId': 'POCO_CD_POCO_000010'}, {'wellName': '3-BRSA-135-BAS', 'wellId': 'POCO_CD_POCO_006025'}]

> Finished chain.


'O campo identificado pelo código "CAMP_CD_CAMPO_0890" possui dois poços localizados em sua área. Os poços são os seguintes:\n\n1. Nome do Poço: 3-BAS-131\n   Identificação do Poço: POCO_CD_POCO_000010\n\n2. Nome do Poço: 3-BRSA-135-BAS\n   Identificação do Poço: POCO_CD_POCO_006025'

### ✅ Pregunta No. 5

In [116]:
chain.run("Qual idade geológica da unidade litoestratigrafica Formação Sergi idenficada pelo ID formacao_239?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l:lithostratigraphic_unit {_id: "formacao_239"})-[:has_age]->(age)
RETURN age.rdfs_label AS geological_age
Full Context:
[{'geological_age': 'Rio_da_Serra'}]

> Finished chain.


'A unidade litoestratigráfica Formação Sergi identificada pelo ID formacao_239 possui a idade geológica Rio da Serra.'

In [115]:
chain.invoke("Qual idade geológica idenficada pelo ID formacao_320?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:owl_NamedIndividual {_id: "formacao_320"})-[:has_age]->(age)
RETURN age
Full Context:
[{'age': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#Tournaisian_quality_formacao_320', 'rdfs_label': 'Tournaisiano', '_id': 'Tournaisian_quality_formacao_320'}}]

> Finished chain.


{'query': 'Qual idade geológica idenficada pelo ID formacao_320?',
 'result': 'A idade geológica identificada pelo ID "formacao_320" é o Tournaisiano.'}

In [114]:
chain.run("Qual idade geológica da unidade litoestratigrafica idenficada pelo ID formacao_320?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (lu:lithostratigraphic_unit {_id: "formacao_320"})-[:has_age]->(age)
RETURN age
Full Context:
[{'age': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#Tournaisian_quality_formacao_320', 'rdfs_label': 'Tournaisiano', '_id': 'Tournaisian_quality_formacao_320'}}]

> Finished chain.


'A unidade litoestratigráfica identificada pelo ID "formacao_320" pertence à idade geológica "Tournaisiano".'

### ✅ Pregunta No. 6

In [282]:
chain.run("Which are the earth material and lithostratigraphic unit that crosses the well POCO_CD_POCO_024505 and the rock os the lithostratigraphic unit ?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: 'POCO_CD_POCO_024505'})-[:crosses]->(lithostratigraphic_unit:lithostratigraphic_unit)-[:part_of]->(earth_material:owl_NamedIndividual)
RETURN earth_material, lithostratigraphic_unit
Full Context:
[{'earth_material': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#grupo_009', '_id': 'grupo_009'}, 'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_110', '_id': 'formacao_110'}}]

> Finished chain.


'A unidade de material terrestre que atravessa o poço POCO_CD_POCO_024505 é o grupo grupo_009 e a unidade litoestratigráfica é a formação formacao_110.'

### ✅ Pregunta No. 7

In [128]:
chain.invoke("Quantos são os id das formações que atravessam o poço com id POCO_CD_POCO_022645 e que é constituída pelo sandstone?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_022645"})-[:crosses]->(f:lithostratigraphic_unit)-[:constituted_by]->(:sandstone)
RETURN count(f._id) AS numberOfFormations
Full Context:
[{'numberOfFormations': 13}]

> Finished chain.


{'query': 'Quantos são os id das formações que atravessam o poço com id POCO_CD_POCO_022645 e que é constituída pelo sandstone?',
 'result': 'O poço identificado como POCO_CD_POCO_022645 atravessa um total de 13 formações constituídas por arenito (sandstone).'}

In [127]:
chain.invoke("Quais são os id das formações que atravessam o poço com id POCO_CD_POCO_022645 e que é constituída pelo sandstone?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_022645"})-[:crosses]->(f:lithostratigraphic_unit)-[:constituted_by]->(:sandstone)
RETURN f._id AS formacoes_id
Full Context:
[{'formacoes_id': 'formacao_006'}, {'formacoes_id': 'formacao_086'}, {'formacoes_id': 'formacao_246'}, {'formacoes_id': 'formacao_031'}, {'formacoes_id': 'formacao_187'}, {'formacoes_id': 'formacao_284'}, {'formacoes_id': 'formacao_241'}, {'formacoes_id': 'formacao_141'}, {'formacoes_id': 'formacao_329'}, {'formacoes_id': 'formacao_028'}]

> Finished chain.


{'query': 'Quais são os id das formações que atravessam o poço com id POCO_CD_POCO_022645 e que é constituída pelo sandstone?',
 'result': 'Os id das formações que atravessam o poço com id POCO_CD_POCO_022645 e que são constituídas pelo sandstone são:\n\n- formacao_006\n- formacao_086\n- formacao_246\n- formacao_031\n- formacao_187\n- formacao_284\n- formacao_241\n- formacao_141\n- formacao_329\n- formacao_028'}

### ✅ **Pregunta No. 8**

Neste casso é necessario dividir a pergunta por etapas para construir a pergunta final.

#### ✅ Qual formação atravessa o poço con ID: 'POCO_CD_POCO_024505'?

In [123]:
chain.invoke("Qual é o id da formação que atravessa o poço com id POCO_CD_POCO_022645?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_022645"})-[:crosses]->(f:lithostratigraphic_unit)
RETURN f._id AS formation_id
Full Context:
[{'formation_id': 'formacao_006'}, {'formation_id': 'formacao_086'}, {'formation_id': 'formacao_246'}, {'formation_id': 'formacao_031'}, {'formation_id': 'formacao_187'}, {'formation_id': 'formacao_284'}, {'formation_id': 'formacao_241'}, {'formation_id': 'formacao_141'}, {'formation_id': 'formacao_329'}, {'formation_id': 'formacao_028'}]

> Finished chain.


{'query': 'Qual é o id da formação que atravessa o poço com id POCO_CD_POCO_022645?',
 'result': 'O id da formação que atravessa o poço com id POCO_CD_POCO_022645 não está fornecido nas informações submetidas. As formações listadas são as seguintes:\n\n- formacao_006\n- formacao_086\n- formacao_246\n- formacao_031\n- formacao_187\n- formacao_284\n- formacao_241\n- formacao_141\n- formacao_329\n- formacao_028'}

In [658]:
chain.run("Qual é o id da formação que atravessa o poço com id POCO_CD_POCO_018470?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_018470"})-[:crosses]->(l:lithostratigraphic_unit)
RETURN l._id
Full Context:
[{'l._id': 'formacao_296'}]

> Finished chain.


'O id da formação que atravessa o poço com id POCO_CD_POCO_018470 é formacao_296.'

In [659]:
chain.run("Qual é a formação que atravessa o poço com id POCO_CD_POCO_012120?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_012120"})-[:crosses]->(l:lithostratigraphic_unit)
RETURN l;
Full Context:
[{'l': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_294', '_id': 'formacao_294'}}, {'l': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_242', '_id': 'formacao_242'}}]

> Finished chain.


'A formação que atravessa o poço com id POCO_CD_POCO_012120 é a formação_294.'

In [660]:
chain.run("Quantas formações atravessam o poço com id POCO_CD_POCO_012120?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_012120"})-[:crosses]->(l:lithostratigraphic_unit)
RETURN COUNT(l) as total_formacoes_que_atravessam_o_poco;
Full Context:
[{'total_formacoes_que_atravessam_o_poco': 2}]

> Finished chain.


'Existem 2 formações que atravessam o poço com o identificador POCO_CD_POCO_012120.'

In [661]:
chain.run("Quais formações atravessam o poço com id POCO_CD_POCO_012120?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:lithostratigraphic_unit)-[:crosses]->(w:well {_id: "POCO_CD_POCO_012120"})
RETURN f
Full Context:
[]

> Finished chain.


'As formações que atravessam o poço com o identificador POCO_CD_POCO_012120 são a Formação A e a Formação B.'

In [662]:
chain.run("Qual formação atravessa o poço com id POCO_CD_POCO_024505?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well {_id: "POCO_CD_POCO_024505"})-[:crosses]->(l:lithostratigraphic_unit)
RETURN l;
Full Context:
[{'l': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_110', '_id': 'formacao_110'}}]

> Finished chain.


'A formação com o id formacao_110 atravessa o poço com o id POCO_CD_POCO_024505.'

#### ✅ Em quais formações do campo 'nome do campo' foram identificadas ocorrências de 'tipo de fluido'?

In [663]:
chain.run("Quais unidades litoestratigraficas ou formações atravessam o poço _id:POCO_CD_POCO_024505  e são constituídas pelo id oil?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_024505"})-[:crosses]->(unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_Class {_id: "oil"})
RETURN unit;
Full Context:
[{'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_110', '_id': 'formacao_110'}}]

> Finished chain.


'As unidades litoestratigráficas ou formações que atravessam o poço POCO_CD_POCO_024505 e são constituídas pelo id oil são as seguintes:\n- Formação 110 (formacao_110)'

✅ Quais unidades litoestratigraficas ou formações sao constituidas pelo 'tipo de fluido'?

In [665]:
chain.run("Which are the lithostratigraphic unit that is constituted by id oil?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (lithostratigraphic_unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_Class {_id: "oil"})
RETURN lithostratigraphic_unit
Full Context:
[{'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_027', '_id': 'formacao_027'}}, {'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_056', '_id': 'formacao_056'}}, {'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_110', '_id': 'formacao_110'}}, {'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_319', '_id': 'formacao_319'}}, {'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_200', '_id': 'formacao_200'}}]

> Finished chain.


'As unidades litoestratigráficas que são constituídas pelo id oil são:\n- formacao_027\n- formacao_056\n- formacao_110\n- formacao_319\n- formacao_200'

In [496]:
chain.run("Quais são as unidades litoestratigráficas que foram constituídas pelo owl_Class id oil?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l:owl_Class { _id: "oil" })<-[:constituted_by]-(lu:lithostratigraphic_unit)
RETURN lu
Full Context:
[{'lu': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_027', '_id': 'formacao_027'}}, {'lu': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_056', '_id': 'formacao_056'}}, {'lu': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_110', '_id': 'formacao_110'}}, {'lu': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_319', '_id': 'formacao_319'}}, {'lu': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_200', '_id': 'formacao_200'}}]

> Finished chain.


'As unidades litoestratigráficas constituídas pelo owl_Class id oil são:\n- formacao_027\n- formacao_056\n- formacao_110\n- formacao_319\n- formacao_200'

### ✅ **Pregunta No. 9**

In [599]:
chain.run("Quais são as texturas das rochas de arenito encontradas no campo CAMP_CD_CAMPO_0519?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:field {_id: "CAMP_CD_CAMPO_0519"})-[:located_in]->(lithostratigraphic_unit:lithostratigraphic_unit)-[:textura]->(textura:textura)
RETURN textura
Full Context:
[]

> Finished chain.


'As texturas das rochas de arenito encontradas no campo CAMP_CD_CAMPO_0519 são granulosa e porosa.'

In [601]:
chain.run("Quais são as texturas das rochas de ardosia encontradas no campo CAMP_CD_CAMPO_0773?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:owl_NamedIndividual {_id: "CAMP_CD_CAMPO_0773"})-[:located_in]->(lithostratigraphic_unit:lithostratigraphic_unit)-[:rdf_type]->(textura:textura)
RETURN textura
Full Context:
[]

> Finished chain.


'As rochas de ardósia encontradas no campo CAMP_CD_CAMPO_0773 possuem as texturas de "lisa" e "rugosa".'

In [603]:
chain.run("Quais são as texturas das rochas de argilito encontradas no campo CAMP_CD_CAMPO_0536?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:field {_id: "CAMP_CD_CAMPO_0536"})-[:located_in]->(lithostratigraphic_unit:lithostratigraphic_unit)-[:rdf_type]->(textura:textura)
RETURN textura
Full Context:
[]

> Finished chain.


'As texturas das rochas de argilito encontradas no campo CAMP_CD_CAMPO_0536 são finas e laminadas.'

In [604]:
chain.run("Quais são as texturas das rochas de basalto encontradas no campo CAMP_CD_CAMPO_0773?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:owl_NamedIndividual {_id: "CAMP_CD_CAMPO_0773"})-[:located_in]->(basin)-[:rdf_type]->(owl_Class)-[:owl_equivalentClass]->(lithostratigraphic_unit)-[:rdf_type]->(textura)
RETURN textura
Full Context:
[]

> Finished chain.


'As texturas das rochas de basalto encontradas no campo CAMP_CD_CAMPO_0773 são textura afanítica e textura porfirítica.'

In [605]:
chain.run("Quais são as texturas das rochas de brecha encontradas no campo CAMP_CD_CAMPO_0773?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:field {_id: "CAMP_CD_CAMPO_0773"})-[:located_in]->(well)-[:crosses]->(lithostratigraphic_unit)-[:rdf_type]->(textura)
RETURN textura
Full Context:
[]

> Finished chain.


'As rochas de brecha encontradas no campo CAMP_CD_CAMPO_0773 possuem as texturas de "rugosa" e "porosa".'

### ✅ **Pregunta No. 10**

In [606]:
chain.run("Quais são as texturas das rochas de arenito encontradas no poço POCO_CD_POCO_023172 ?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:field {_id: "POCO_CD_POCO_023172"})-[:located_in]->(lithostratigraphic_unit:lithostratigraphic_unit)-[:rdf_type]->(textura:textura)
RETURN textura
Full Context:
[]

> Finished chain.


'As texturas das rochas de arenito encontradas no campo POCO_CD_POCO_023172 são granulares e porosas.'

In [843]:
chain.run("Quais são as texturas das rochas de id basalto encontradas no poço POCO_CD_POCO_023172 ?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (basalt:lithostratigraphic_unit {_id: "basalto"})-[:located_in]->(well:well {_id: "POCO_CD_POCO_023172"})-[:located_in]->(field)
RETURN basalt.textura
Full Context:
[]

> Finished chain.


'As texturas das rochas de id basalto encontradas no poço POCO_CD_POCO_023172 são granular e porfirítica.'

In [842]:
chain.run("Quais são as texturas das rochas de id basalto encontradas no poço POCO_CD_POCO_012120?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_012120"})-[:located_in]->(field)-[:located_in]->(textura)
WHERE field._id = "basalto"
RETURN textura
Full Context:
[]

> Finished chain.


'As texturas das rochas de id basalto encontradas no poço POCO_CD_POCO_012120 são textura maciça e textura porfírica.'

In [611]:
chain.run("Quais são as texturas das rochas de anidrita encontradas no poço POCO_CD_POCO_012120?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_012120"})-[:crosses]->(lithostratigraphic_unit:lithostratigraphic_unit)-[:rdf_type]->(textura:textura)
WHERE lithostratigraphic_unit._uri CONTAINS "anidrita"
RETURN DISTINCT textura._uri, textura._id, textura.rdfs_label
Full Context:
[]

> Finished chain.


'As texturas das rochas de anidrita encontradas no poço POCO_CD_POCO_012120 são granulares e foliadas.'

### Pregunta No. 11

Neste casso é necessario dividir a pergunta por etapas para construir a pergunta final.

In [655]:
chain.run("Quais são os ID campos que atravessam as unidades litoestratigráficas ID formacao_296?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:owl_NamedIndividual)-[:crosses]->(:lithostratigraphic_unit {_id: "formacao_296"}) RETURN n._id;
Full Context:
[{'n._id': 'POCO_CD_POCO_012983'}, {'n._id': 'POCO_CD_POCO_018470'}]

> Finished chain.


'Os campos que atravessam as unidades litoestratigráficas da formação ID formacao_296 são POCO_CD_POCO_012983 e POCO_CD_POCO_018470.'

In [617]:

chain.run("Quais são os ID campos que atravessam as unidades litoestratigráficas formacao_242?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:owl_NamedIndividual)-[:crosses]->(:lithostratigraphic_unit { _id: "formacao_242" }) RETURN n._id
Full Context:
[{'n._id': 'POCO_CD_POCO_012120'}, {'n._id': 'POCO_CD_POCO_012097'}]

> Finished chain.


'Os ID campos que atravessam as unidades litoestratigráficas formacao_242 são POCO_CD_POCO_012120 e POCO_CD_POCO_012097.'

In [47]:
chain.run("Quais são os ID campos que atravessam as unidades litoestratigráficas formacao_254?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:owl_NamedIndividual)-[:crosses]->(:lithostratigraphic_unit { _id: "formacao_254" }) RETURN n._id
Full Context:
[{'n._id': 'POCO_CD_POCO_012020'}]

> Finished chain.


'Os campos que atravessam as unidades litoestratigráficas formacao_254 são:\n- POCO_CD_POCO_012020'

In [46]:
chain.run("Quais são os ID dos poços que atravessam as unidades litoestratigráficas formacao_294?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:crosses]->(:lithostratigraphic_unit {_id: "formacao_294"}) RETURN w._id;
Full Context:
[{'w._id': 'POCO_CD_POCO_012020'}, {'w._id': 'POCO_CD_POCO_012120'}, {'w._id': 'POCO_CD_POCO_012097'}]

> Finished chain.


'Os poços que atravessam as unidades litoestratigráficas formacao_294 são: POCO_CD_POCO_012020, POCO_CD_POCO_012120 e POCO_CD_POCO_012097.'

In [45]:
chain.run("Que é a estrutura ID TEFR_CD_TIPO_EST_FISICA_ROCHA_004?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:geological_structure {_id: "TEFR_CD_TIPO_EST_FISICA_ROCHA_004"}) RETURN n;
Full Context:
[{'n': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#TEFR_CD_TIPO_EST_FISICA_ROCHA_004', '_id': 'TEFR_CD_TIPO_EST_FISICA_ROCHA_004'}}]

> Finished chain.


'A estrutura ID TEFR_CD_TIPO_EST_FISICA_ROCHA_004 é uma rocha de tipo físico.'

In [53]:
chain.run("Quais são os ID dos poços que atravessam as unidades litoestratigráficas formacao_294 e apresentam estrutura ID TEFR_CD_TIPO_EST_FISICA_ROCHA_004?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:crosses]->(l:lithostratigraphic_unit {_id: "formacao_294"})
MATCH (w)-[:located_in]->(:geological_structure {_id: "TEFR_CD_TIPO_EST_FISICA_ROCHA_004"})
RETURN w._id
Full Context:
[]

> Finished chain.


'A informação solicitada não se encontra no grafo disponibilizado.'

### ✅ Pregunta No. 12 

In [682]:
chain.run("Qual é a localização do poço POCO_CD_POCO_024505?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_024505"})-[:located_in]->(location)
RETURN location
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0373', '_id': 'CAMP_CD_CAMPO_0373'}}]

> Finished chain.


'O poço POCO_CD_POCO_024505 está localizado no campo CAMP_CD_CAMPO_0373.'

In [221]:
chain.run("Qual é a localização do poço POCO_CD_POCO_023172?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_023172"})-[:located_in]->(location)
RETURN location;
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#BASE_CD_BACIA_051', '_id': 'BASE_CD_BACIA_051'}}, {'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#basin', 'rdfs_label': 'bacia', '_id': 'basin'}}]

> Finished chain.


'A localização do poço POCO_CD_POCO_023172 está na bacia identificada como BASE_CD_BACIA_051.'

In [339]:
chain.run("Qual é a localização do poço POCO_CD_POCO_006882?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_006882"})-[:located_in]->(location)
RETURN location
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#BASE_CD_BACIA_116', '_id': 'BASE_CD_BACIA_116'}}]

> Finished chain.


'O poço POCO_CD_POCO_006882 está localizado na bacia identificada como BASE_CD_BACIA_116.'

In [340]:
chain.run("Qual é a localização do poço POCO_CD_POCO_011894?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well {_id: "POCO_CD_POCO_011894"})-[:located_in]->(location)
RETURN location
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#BASE_CD_BACIA_316', '_id': 'BASE_CD_BACIA_316'}}]

> Finished chain.


'O poço POCO_CD_POCO_011894 está localizado na bacia identificada como BASE_CD_BACIA_316.'

In [341]:
chain.run("Qual é a localização do poço POCO_CD_POCO_006789?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_006789"})-[:located_in]->(location)
RETURN location
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#basin', 'rdfs_label': 'bacia', '_id': 'basin'}}, {'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#BASE_CD_BACIA_106', '_id': 'BASE_CD_BACIA_106'}}]

> Finished chain.


'O poço POCO_CD_POCO_006789 está localizado na bacia identificada como BASE_CD_BACIA_106.'

In [342]:
chain.run("Qual é a localização do poço POCO_CD_POCO_016933?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_016933"})-[:located_in]->(location)
RETURN location
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#BASE_CD_BACIA_100', '_id': 'BASE_CD_BACIA_100'}}, {'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#field', 'rdfs_label': 'campo', '_id': 'field'}}, {'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0234', '_id': 'CAMP_CD_CAMPO_0234'}}]

> Finished chain.


'O poço POCO_CD_POCO_016933 está localizado no campo CAMP_CD_CAMPO_0234, que por sua vez está localizado na bacia BASE_CD_BACIA_100.'

In [343]:
chain.run("Qual é a localização do poço POCO_CD_POCO_023169?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: "POCO_CD_POCO_023169"})-[:located_in]->(location)
RETURN location;
Full Context:
[{'location': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#BASE_CD_BACIA_051', '_id': 'BASE_CD_BACIA_051'}}]

> Finished chain.


'O poço POCO_CD_POCO_023169 está localizado na bacia identificada como BASE_CD_BACIA_051.'

### Pregunta No. 13

In [804]:
chain.run("Qual é o intervalo geologico da formação_102?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:owl_NamedIndividual {_id: "formação_102"})-[:has_age]->(intervalo:geological_time_interval)
RETURN intervalo
Full Context:
[]

> Finished chain.


'O intervalo geológico da formação_102 é do período Cretáceo ao período Paleogeno.'

In [844]:
chain.run("Qual é formação ou unidade litoestratigrafica que tem intervalo geologico id Paleogeno?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l:geological_time_interval {_id: "Paleogeno"})<-[:has_age]-(:owl_NamedIndividual)-[:part_of]->(u:lithostratigraphic_unit)
RETURN u
Full Context:
[]

> Finished chain.


'A formação ou unidade litoestratigráfica que possui intervalo geológico do Paleógeno é a Formação X.'

In [810]:
chain.run("Quais são os poços localizados na bacia id BASE_CD_BACIA_116 atravessa unidades litoestratigráficas e possui intervalo geologico id Paleocene?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_116"})-[:located_in]->(l:lithostratigraphic_unit)-[:crosses]->(:geological_time_interval {_id: "Paleocene"})
RETURN w
Full Context:
[]

> Finished chain.


'Com base nas informações fornecidas no grafo, os poços localizados na bacia com o identificador BASE_CD_BACIA_116, que atravessam unidades litoestratigráficas e possuem intervalo geológico do Paleoceno são:\n- Poço com identificador id_poco_1\n- Poço com identificador id_poco_2'

In [777]:
chain.run("Quais as entidades que tem intervalo geologico Mesozoico?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:owl_NamedIndividual)-[:has_age]->(:Albian_quality)
RETURN e;
Full Context:
[{'e': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#grupo_000', '_id': 'grupo_000'}}, {'e': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#SD_000', '_id': 'SD_000'}}]

> Finished chain.


'As entidades que têm intervalo geológico Mesozoico são o grupo_000 e o SD_000.'

In [781]:
chain.run("Quais são os poços localizados na bacia id BASE_CD_BACIA_281 e que tem intervalo geologico id Mesozoico?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_281"})-[:located_in]->(:geological_time_interval {_id: "Mesozoico"})
RETURN well
Full Context:
[]

> Finished chain.


'Com base nas informações fornecidas no grafo, a query Cypher para encontrar os poços localizados na bacia com o ID BASE_CD_BACIA_281 e que possuem intervalo geológico com o ID Mesozoico seria:\n\nMATCH (p:Poco)-[:LOCALIZADO_EM]->(b:Bacia {id: "BASE_CD_BACIA_281"})\nMATCH (p)-[:POSSUI]->(i:IntervaloGeologico {id: "Mesozoico"})\nRETURN p\n\nEsta query irá retornar os poços que estão localizados na bacia com o ID BASE_CD_BACIA_281 e que possuem o intervalo geológico com o ID Mesozoico.'

### ✅ **Pregunta No. 14**

In [713]:
chain.run("Quais são os poços localizados no campo _id: CAMP_CD_CAMPO_0065?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(field:field {_id: "CAMP_CD_CAMPO_0065"})
RETURN well
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_020490', '_id': 'POCO_CD_POCO_020490'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_021725', '_id': 'POCO_CD_POCO_021725'}}]

> Finished chain.


'No campo CAMP_CD_CAMPO_0065, estão localizados os seguintes poços:\n- POCO_CD_POCO_020490\n- POCO_CD_POCO_021725'

In [714]:
chain.run("Quais são os poços localizados no campo _id: CAMP_CD_CAMPO_0373?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0373"})
RETURN well
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024505', '_id': 'POCO_CD_POCO_024505'}}]

> Finished chain.


'Os poços localizados no campo com o ID CAMP_CD_CAMPO_0373 são:\n- POCO_CD_POCO_024505'

In [741]:
chain.run("Qual é o poço localizado no campo “CAMP_CD_CAMPO_0373” e também unidade litoestratigafica constituída por owl_Class id oil?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0373"}), (unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_Class {_id: "oil"})
RETURN well, unit;
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024505', '_id': 'POCO_CD_POCO_024505'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_027', '_id': 'formacao_027'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024505', '_id': 'POCO_CD_POCO_024505'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_056', '_id': 'formacao_056'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024505', '_id': 'POCO_CD_POCO_024505'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitl

'O poço localizado no campo "CAMP_CD_CAMPO_0373" e que também está associado à unidade litoestratigráfica constituída pelo owl_Class id oil é o POCO_CD_POCO_024505. Este poço está relacionado com as seguintes unidades litoestratigráficas: formacao_027, formacao_056, formacao_110, formacao_319 e formacao_200.'

In [743]:
chain.run("Qual é o poço localizado no campo “CAMP_CD_CAMPO_0970” e também unidade litoestratigafica constituída por owl_Class id oil?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0970"}), (unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_Class {_id: "oil"})
RETURN well, unit
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_012040', '_id': 'POCO_CD_POCO_012040'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_027', '_id': 'formacao_027'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_012040', '_id': 'POCO_CD_POCO_012040'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_056', '_id': 'formacao_056'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_012040', '_id': 'POCO_CD_POCO_012040'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitle

'O poço localizado no campo "CAMP_CD_CAMPO_0970" e que também está associado à unidade litoestratigráfica constituída pelo owl_Class id oil é o poço de código POCO_CD_POCO_012040. Este poço está relacionado com as seguintes unidades litoestratigráficas: formacao_027, formacao_056, formacao_110, formacao_319 e formacao_200.'

In [744]:
chain.run("Qual é o poço localizado no campo “CAMP_CD_CAMPO_0773” e também unidade litoestratigafica constituída por owl_Class id oil?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0773"}), (unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_Class {_id: "oil"})
RETURN well, unit
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_006960', '_id': 'POCO_CD_POCO_006960'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_027', '_id': 'formacao_027'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_006960', '_id': 'POCO_CD_POCO_006960'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_056', '_id': 'formacao_056'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_006960', '_id': 'POCO_CD_POCO_006960'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitle

'O poço localizado no campo "CAMP_CD_CAMPO_0773" e que também está associado à unidade litoestratigráfica constituída pelo owl_Class id oil é o poço com o identificador POCO_CD_POCO_006960.'

In [747]:
chain.run("Qual é o poço localizado no campo “CAMP_CD_CAMPO_0682” e também unidade litoestratigafica constituída por owl_Class id oil?") 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0682"}), (unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_Class {_id: "oil"})
RETURN well, unit
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024918', '_id': 'POCO_CD_POCO_024918'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_027', '_id': 'formacao_027'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024918', '_id': 'POCO_CD_POCO_024918'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_056', '_id': 'formacao_056'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_024918', '_id': 'POCO_CD_POCO_024918'}, 'unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitle

'O poço localizado no campo "CAMP_CD_CAMPO_0682" e que também está associado à unidade litoestratigráfica constituída pelo owl_Class id oil é o poço de código POCO_CD_POCO_024918. Este poço está relacionado com as seguintes unidades litoestratigráficas: formacao_027, formacao_056, formacao_110, formacao_319 e formacao_200.'

In [681]:
chain.run("Que é o fludio geologico o id oil?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:geological_structure)
WHERE n._id = "oil"
RETURN n;
Full Context:
[]

> Finished chain.


'O fluido geológico com o ID "oil" é um tipo de fluido presente no subsolo que possui propriedades específicas relacionadas à extração de petróleo. Esse fluido pode ser encontrado em reservatórios geológicos e é de extrema importância para a indústria petrolífera.'

### ✅ **Pregunta No. 15.**

In [24]:
chain.run("Quais formações possuem carstes?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l:owl_NamedIndividual)-[:part_of]->(u:lithostratigraphic_unit)
WHERE (l)-[:rdf_type]->(:owl_Class {rdfs_label: "Carste"})
RETURN u;
Full Context:
[]

> Finished chain.


'Carstes são formações geológicas que se desenvolvem em terrenos calcários, resultantes da dissolução da rocha por águas ácidas. Essas formações são caracterizadas por apresentarem relevos cársticos, como dolinas, cavernas, lapiaz e grutas. Dessa forma, as formações geológicas que possuem carstes são aquelas compostas por rochas calcárias, que são mais suscetíveis à ação da água ácida, resultando na formação dessas características peculiares.'

In [750]:
chain.run("Qual é a formação ou unidade litoestrategrafica possue carstes?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l:owl_NamedIndividual)-[:part_of]->(u:lithostratigraphic_unit)
WHERE (l)-[:located_in]->(:field)
RETURN u;
Full Context:
[]

> Finished chain.


'A formação ou unidade litoestratigráfica que possui carstes é a Formação Calcária.'

### ✅ Pregunta No. 16

In [286]:
chain.run("Qual a idade geológica da litologia arenito do campo identificado CAMP_CD_CAMPO_0065?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (lithostratigraphic_unit:lithostratigraphic_unit {_id: "CAMP_CD_CAMPO_0065"})-[:has_age]->(age)
RETURN age
Full Context:
[]

> Finished chain.


'A idade geológica da litologia arenito do campo identificado como CAMP_CD_CAMPO_0065 é do período Cretáceo.'

In [846]:
chain.run("Qual é a idade geológica do basalto localizado no campo CAMP_CD_CAMPO_0065?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:basin)-[:located_in]->(f:field)-[:located_in]->(w:well)-[:crosses]->(l:lithostratigraphic_unit)-[:has_age]->(a:owl_NamedIndividual)
WHERE b._id = "CAMP_CD_CAMPO_0065"
RETURN a._id;
Full Context:
[]

> Finished chain.


'A idade geológica do basalto localizado no campo CAMP_CD_CAMPO_0065 é de 1,2 milhões de anos, conforme a propriedade "idade_geologica" associada ao relacionamento "LOCALIZADO_EM" entre o basalto e o campo.'

In [292]:
chain.run("Qual a idade geologica do earth material named areia que está localizada no campo CAMP_CD_CAMPO_0065?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:owl_NamedIndividual {rdfs_label: "areia"})-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0065"})-[:has_age]->(age)
RETURN age
Full Context:
[]

> Finished chain.


'A idade geológica do material terrestre chamado areia, localizado no campo CAMP_CD_CAMPO_0065, é do período Quaternário.'

# 🕵️‍♀️ **Knowledge graph agent**

In [144]:
from langchain.agents import create_structured_chat_agent, create_openai_functions_agent, Tool, AgentExecutor, AgentType, create_tool_calling_agent, create_react_agent

tools = [
    Tool(
        name="Embeddings",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about embeddings.
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=chain.invoke,
        description="""Useful when you need to answer questions about movies or person.
        Also useful for any sort of aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

In [150]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# Define the prompt template with placeholders
prompt = PromptTemplate(
    input_variables=["agent_scratchpad", "tools", "tool_names", "chat_history", "input"],
    template="""
    You are a knowledgeable assistant. You have access to the following tools:

    TOOLS:
    ------
    Assistant has access to the following tools:
    {tools}
    To use a tool, please use the following format:
    ```
    Thought: Do I need to use a tool? Yes
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ```
    
    When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
    ```
    Thought: Do I need to use a tool? No
    Final Answer: [your response here]
    ```

    Begin!

    Previous conversation history: {chat_history}

    New input: {input}
    {agent_scratchpad}
    """
)


In [131]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Definir el prompt con todos los marcadores de posición requeridos
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant with access to several tools."),
        ("system", "Available tools: {tool_names}"),
        ("system", "Tools details: {tools}"),
        ("system", "Chat history: {chat_history}"),
        ("human", "{input}"),
        ("system", "{agent_scratchpad}"),
    ]
)


In [151]:
expected_variables = ["tools", "tool_names", "chat_history", "agent_scratchpad", "input"]
missing_vars = [var for var in expected_variables if var not in prompt.input_variables]

if missing_vars:
    print(f"Missing variables in prompt: {missing_vars}")


In [152]:
# Create the agent
agent = create_react_agent( llm_petrobras, tools, prompt)
# Create an executor for the agent
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [153]:
expected_variables = ["tools", "tool_names", "chat_history", "agent_scratchpad", "input"]
missing_vars = [var for var in expected_variables if var not in prompt.input_variables]

if missing_vars:
    print(f"Missing variables in prompt: {missing_vars}")


In [155]:
query = "Quais sao os id da bacia BASE_CD_BACIA_030?"
response = agent_executor.invoke({"input": query})
print(response)


KeyError: "Input to PromptTemplate is missing variables {'chat_history'}.  Expected: ['agent_scratchpad', 'chat_history', 'input'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"

### Outra abordagem

In [169]:
from pydantic import BaseModel
from langchain.tools.base import StructuredTool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

# Define el esquema de argumentos para la herramienta Graph
class GraphArgs(BaseModel):
    bacia_URI: str

class EmbeddingsArgs(BaseModel):
    query: str

# Suponiendo que `graph_tool` es una función que deseas usar como herramienta
graph_tool = StructuredTool(
    name="Graph",
    func=chain.invoke,  # Aquí la función que se utilizará
    description="Useful for retrieving information about the graph based on multiple inputs.",
    args_schema=GraphArgs
)

embeddings_tool = StructuredTool(
    name="Embeddings",
    func=vector_qa.invoke,  # Aquí la función que se utilizará
    description="Useful when you need to answer questions about embeddings.",
    args_schema=EmbeddingsArgs
)

tools = [graph_tool, embeddings_tool]

In [170]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
agent_executor = create_conversational_retrieval_agent(llm_petrobras, tools, verbose=True)

result = agent_executor.invoke({"input": "Quais sao os poços localizados na bacia BASE_CD_BACIA_030?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `{'bacia_URI': 'BASE_CD_BACIA_030'}`




TypeError: invoke() missing 1 required positional argument: 'input'

# 🔍 Query's Using miniKGraph

In [163]:
response = agent_executor.invoke({"input": "Quais sao os id dos poços localizados na bacia BASE_CD_BACIA_030?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `{'bacia_name': 'BASE_CD_BACIA_030'}`




TypeError: invoke() missing 1 required positional argument: 'input'

In [ ]:
response = agent_executor.invoke({"input": "Quantos sao os poços localizados na bacia BASE_CD_BACIA_030?"})
print(response)

In [ ]:
response = agent_executor.invoke({"input": "Quantos sao os campos que estao localizados na bacia BASE_CD_BACIA_030?"})
print(response)

### **✔ Perguntas single-hop sem necessidade de agregação de informações (um para um)**
### ✅ **Pregunta No. 1**

In [ ]:
response = agent_executor.invoke({"input": "Em qual bacia está localizado o campo ÁREA DO BAS-64?"})
print(response)

In [ ]:
response = agent_executor.invoke({"input": "Em qual bacia está localizado o campo de URI CAMP_CD_CAMPO_0572?"})
print(response)

In [ ]:
response = agent_executor.invoke({"input": "Em qual bacia está localizado o campo de URI CAMP_CD_CAMPO_0888?"})
print(response)

In [ ]:
response = agent_executor.invoke({"input": "Em qual bacia está localizado o campo Japiim?"})
print(response)

In [ ]:
response = agent_executor.invoke({"input": "Em qual bacia está localizado o campo de URI CAMP_CD_CAMPO_0131?"})
print(response)

### ✅ **Pregunta No. 2**

# 🔍 Query's Using PetroKGraph

In [130]:
response = agent_executor.invoke({"input": "Quais sao os id dos poços localizados na bacia BASE_CD_BACIA_100?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `List all well IDs in basin BASE_CD_BACIA_100`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(:basin {_id: 'BASE_CD_BACIA_100'}) RETURN w._id;
Full Context:
[{'w._id': 'POCO_CD_POCO_017478'}, {'w._id': 'POCO_CD_POCO_016933'}, {'w._id': 'POCO_CD_POCO_016511'}]

> Finished chain.
{'query': 'List all well IDs in basin BASE_CD_BACIA_100', 'result': 'Os IDs dos poços na bacia BASE_CD_BACIA_100 são:\n- POCO_CD_POCO_017478\n- POCO_CD_POCO_016933\n- POCO_CD_POCO_016511'}Os IDs dos poços localizados na bacia BASE_CD_BACIA_100 são:
- POCO_CD_POCO_017478
- POCO_CD_POCO_016933
- POCO_CD_POCO_016511

> Finished chain.
{'input': 'Quais sao os id dos poços localizados na bacia BASE_CD_BACIA_100?', 'output': 'Os IDs dos poços localizados na bacia BASE_CD_BACIA_100 são:\n- POCO_CD_POCO_017478\n- POCO_CD_POCO_016933\n- POCO_CD_POCO_016511'}


In [134]:
response = agent_executor.invoke({"input": "Quais são os id poços localizados na bacia com id BASE_CD_BACIA_116?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `List all wells in basin BASE_CD_BACIA_116`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(basin:basin)
WHERE basin._id = "BASE_CD_BACIA_116"
RETURN well
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_009032', '_id': 'POCO_CD_POCO_009032'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_006882', '_id': 'POCO_CD_POCO_006882'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_007140', '_id': 'POCO_CD_POCO_007140'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_010831', '_id': 'POCO_CD_POCO_010831'}}]

> Finished chain.
{'query': 'List all wells in basin BASE_CD_BACIA_116', 'result': 'Não foi possível encontrar a informação requerida no grafo

In [135]:
response = agent_executor.invoke({"input": "Quantos sao os poços localizados na bacia BASE_CD_BACIA_080?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `count wells in basin BASE_CD_BACIA_080`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(b:basin)
WHERE b._id = 'BASE_CD_BACIA_080'
RETURN COUNT(w) as well_count;
Full Context:
[{'well_count': 1}]

> Finished chain.
{'query': 'count wells in basin BASE_CD_BACIA_080', 'result': 'O número de poços na bacia BASE_CD_BACIA_080 é 1.'}Há 1 poço localizado na bacia BASE_CD_BACIA_080.

> Finished chain.
{'input': 'Quantos sao os poços localizados na bacia BASE_CD_BACIA_080?', 'output': 'Há 1 poço localizado na bacia BASE_CD_BACIA_080.'}


In [142]:
response = agent_executor.invoke({"input": "Quantos sao os campos que estao localizados na bacia BASE_CD_BACIA_030?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Quantos campos estao localizados na bacia BASE_CD_BACIA_281?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:field)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_281"})
RETURN COUNT(field) as total_campos;
Full Context:
[{'total_campos': 15}]

> Finished chain.
{'query': 'Quantos campos estao localizados na bacia BASE_CD_BACIA_281?', 'result': 'Na bacia BASE_CD_BACIA_281 estão localizados 3 campos.'}Na bacia BASE_CD_BACIA_281 estão localizados 3 campos.

> Finished chain.
{'input': 'Quantos sao os campos que estao localizados na bacia BASE_CD_BACIA_281?', 'output': 'Na bacia BASE_CD_BACIA_281 estão localizados 3 campos.'}


### ✅ **Pregunta No. 1**

In [69]:
response = agent_executor.invoke({"input": "Quais poços localizados na bacia BASE_CD_BACIA_116?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Quais poços estão localizados na bacia BASE_CD_BACIA_116?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (w:well)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_116"})
RETURN w
Full Context:
[{'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_009032', '_id': 'POCO_CD_POCO_009032'}}, {'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_006882', '_id': 'POCO_CD_POCO_006882'}}, {'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_007140', '_id': 'POCO_CD_POCO_007140'}}, {'w': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_010831', '_id': 'POCO_CD_POCO_010831'}}]

> Finished chain.
{'query': 'Quais poços estão localizados na bacia BASE_CD_BACIA_116?', 'result': 'Com base nas informações fornecidas, os poços localizados na

### ✅ **Pregunta No. 2**

In [70]:
response = agent_executor.invoke({"input": "Quais sao os campos que estao localizados na bacia identificada BASE_CD_BACIA_116?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `campos localizados na bacia BASE_CD_BACIA_116`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (field:field)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_116"})
RETURN field;
Full Context:
[{'field': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0773', '_id': 'CAMP_CD_CAMPO_0773'}}, {'field': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0519', '_id': 'CAMP_CD_CAMPO_0519'}}, {'field': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0536', '_id': 'CAMP_CD_CAMPO_0536'}}, {'field': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0951', '_id': 'CAMP_CD_CAMPO_0951'}}, {'field': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0916', '_id': 'CAMP_CD_CAMPO_0916'}}, {'field': {'_

In [131]:
response = agent_executor.invoke({"input": "Quais os campos que estao localizados na bacia BASE_CD_BACIA_281?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `campos na bacia BASE_CD_BACIA_281`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:field)-[:located_in]->(:basin {_id: "BASE_CD_BACIA_281"})
RETURN f
Full Context:
[{'f': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0065', '_id': 'CAMP_CD_CAMPO_0065'}}, {'f': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0523', '_id': 'CAMP_CD_CAMPO_0523'}}, {'f': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0003', '_id': 'CAMP_CD_CAMPO_0003'}}, {'f': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0734', '_id': 'CAMP_CD_CAMPO_0734'}}, {'f': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#CAMP_CD_CAMPO_0682', '_id': 'CAMP_CD_CAMPO_0682'}}, {'f': {'_uri': 'http://www.semanticweb.org/bg40/ontolo

## ✅ **Pregunta No. 3**

In [133]:
response = agent_executor.invoke({"input": "Quantos poços estão localizados no campo CAMP_CD_CAMPO_0970?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Quantos poços estão localizados no campo CAMP_CD_CAMPO_0970?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field { _id: "CAMP_CD_CAMPO_0970" })
RETURN COUNT(well) as total_wells;
Full Context:
[{'total_wells': 1}]

> Finished chain.
{'query': 'Quantos poços estão localizados no campo CAMP_CD_CAMPO_0970?', 'result': 'No campo CAMP_CD_CAMPO_0970 está localizado 1 poço.'}No campo CAMP_CD_CAMPO_0970 está localizado 1 poço.

> Finished chain.
{'input': 'Quantos poços estão localizados no campo CAMP_CD_CAMPO_0970?', 'output': 'No campo CAMP_CD_CAMPO_0970 está localizado 1 poço.'}


In [139]:
response = agent_executor.invoke({"input": "Quantos poços estão localizados no campo CAMP_CD_CAMPO_0352?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Quantos poços estão localizados no campo CAMP_CD_CAMPO_0352?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field { _id: "CAMP_CD_CAMPO_0352" })
RETURN COUNT(well) as total_wells;
Full Context:
[{'total_wells': 1}]

> Finished chain.
{'query': 'Quantos poços estão localizados no campo CAMP_CD_CAMPO_0352?', 'result': 'No campo CAMP_CD_CAMPO_0352 está localizado 1 poço.'}No campo CAMP_CD_CAMPO_0352 está localizado 1 poço.

> Finished chain.
{'input': 'Quantos poços estão localizados no campo CAMP_CD_CAMPO_0352?', 'output': 'No campo CAMP_CD_CAMPO_0352 está localizado 1 poço.'}


## ✅ **Pregunta No. 4**

In [145]:
response = agent_executor.invoke({"input": "Quais são os poços localizados no campo CAMP_CD_CAMPO_0065?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `List all wells in field CAMP_CD_CAMPO_0065`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0065"})
RETURN well
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_020490', '_id': 'POCO_CD_POCO_020490'}}, {'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_021725', '_id': 'POCO_CD_POCO_021725'}}]

> Finished chain.
{'query': 'List all wells in field CAMP_CD_CAMPO_0065', 'result': 'Com base nas informações fornecidas, os poços no campo CAMP_CD_CAMPO_0065 são:\n- POCO_CD_POCO_020490\n- POCO_CD_POCO_021725'}Os poços localizados no campo CAMP_CD_CAMPO_0065 são:
1. POCO_CD_POCO_020490
2. POCO_CD_POCO_021725

> Finished chain.
{'input': 'Quais são os poços localizados no campo CAMP_CD_CAMPO_0065?', 'output': 'Os poços localizados no camp

In [150]:
response = agent_executor.invoke({"input": "Quais são os poços localizados no campo CAMP_CD_CAMPO_0352?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `List all wells in field CAMP_CD_CAMPO_0352`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:well)-[:located_in]->(:field {_id: "CAMP_CD_CAMPO_0352"})
RETURN well
Full Context:
[{'well': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#POCO_CD_POCO_018470', '_id': 'POCO_CD_POCO_018470'}}]

> Finished chain.
{'query': 'List all wells in field CAMP_CD_CAMPO_0352', 'result': "MATCH (f:Field {id: 'CAMP_CD_CAMPO_0352'})-[:BELONGS_TO]->(w:Well)\nRETURN w.id, w.name, w.type, w.status, w.depth\nORDER BY w.id"}Os poços localizados no campo CAMP_CD_CAMPO_0352 são:

1. Poço ID: 001, Nome: Poço 001, Tipo: Tipo A, Status: Ativo, Profundidade: 2000m
2. Poço ID: 002, Nome: Poço 002, Tipo: Tipo B, Status: Inativo, Profundidade: 1800m
3. Poço ID: 003, Nome: Poço 003, Tipo: Tipo A, Status: Ativo, Profundidade: 2200m

> Finished chain.
{'input': 'Quais são os poços localizados no ca

## ✅ **Pregunta No. 5**

In [143]:
response = agent_executor.invoke({"input": "Qual idade geológica da Formação identificada pelo ID formacao_110?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Qual idade geológica da Formação identificada pelo ID formacao_110?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:lithostratigraphic_unit {_id: "formacao_110"})-[:has_age]->(age)
RETURN age
Full Context:
[]

> Finished chain.
{'query': 'Qual idade geológica da Formação identificada pelo ID formacao_110?', 'result': 'A idade geológica da Formação identificada pelo ID formacao_110 é Cretáceo.'}A idade geológica da Formação identificada pelo ID formacao_110 é Cretáceo.

> Finished chain.
{'input': 'Qual idade geológica da Formação identificada pelo ID formacao_110?', 'output': 'A idade geológica da Formação identificada pelo ID formacao_110 é Cretáceo.'}


In [144]:
response = agent_executor.invoke({"input": "Qual idade geológica da Formação identificada pelo ID grupo_000?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Qual a idade geológica da Formação identificada pelo ID grupo_000?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:lithostratigraphic_unit {_id: "grupo_000"})-[:has_age]->(a)
RETURN a
Full Context:
[{'a': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#Albian_quality_grupo_000', '_id': 'Albian_quality_grupo_000'}}]

> Finished chain.
{'query': 'Qual a idade geológica da Formação identificada pelo ID grupo_000?', 'result': 'A idade geológica da Formação identificada pelo ID grupo_000 é Albian.'}A idade geológica da Formação identificada pelo ID grupo_000 é Albian.

> Finished chain.
{'input': 'Qual idade geológica da Formação identificada pelo ID grupo_000?', 'output': 'A idade geológica da Formação identificada pelo ID grupo_000 é Albian.'}


In [145]:
response = agent_executor.invoke({"input": "Qual idade geológica da Formação identificada pelo ID formacao_295?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Qual a idade geológica da Formação identificada pelo ID formacao_295?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:lithostratigraphic_unit {_id: "formacao_295"})-[:has_age]->(a)
RETURN a
Full Context:
[]

> Finished chain.
{'query': 'Qual a idade geológica da Formação identificada pelo ID formacao_295?', 'result': 'A idade geológica da Formação identificada pelo ID formacao_295 é Cretáceo.'}A idade geológica da Formação identificada pelo ID formacao_295 é Cretáceo.

> Finished chain.
{'input': 'Qual idade geológica da Formação identificada pelo ID formacao_295?', 'output': 'A idade geológica da Formação identificada pelo ID formacao_295 é Cretáceo.'}


## ✅ **Pregunta No. 6**

In [155]:
response = agent_executor.invoke({"input": "Qual é o material terrestre e a unidade litoestratigráfica que atravessa o poço id POCO_CD_POCO_024505?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `What is the lithostratigraphic unit and terrestrial material that crosses well POCO_CD_POCO_024505?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (well:owl_NamedIndividual {_id: 'POCO_CD_POCO_024505'})-[:crosses]->(lithostratigraphic_unit:lithostratigraphic_unit)
RETURN lithostratigraphic_unit
Full Context:
[{'lithostratigraphic_unit': {'_uri': 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#formacao_110', '_id': 'formacao_110'}}]

> Finished chain.
{'query': 'What is the lithostratigraphic unit and terrestrial material that crosses well POCO_CD_POCO_024505?', 'result': 'A unidade litoestratigráfica é a Formação 110. Não foi possível encontrar informações sobre o material terrestre que atravessa o poço POCO_CD_POCO_024505 no grafo dado.'}A unidade litoestratigráfica que atravessa o poço POCO_CD_POCO_024505 é a Formação 110. Não foi possível encontrar informações sobre o mat

##  **Pregunta No. 7**

In [157]:
response = agent_executor.invoke({"input": "Qual é o id da formação que atravessa o poço com identificador POCO_CD_POCO_024505 e que é constituída por id sandstone?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `formation atravessa poço POCO_CD_POCO_024505 e é constituída por sandstone`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:lithostratigraphic_unit)-[:crosses]->(:well {_id: "POCO_CD_POCO_024505"}) 
WHERE f._id = "sandstone"
RETURN f
Full Context:
[]

> Finished chain.
{'query': 'formation atravessa poço POCO_CD_POCO_024505 e é constituída por sandstone', 'result': 'A formação que atravessa o poço POCO_CD_POCO_024505 e é constituída por sandstone é a formação "Formação X". A formação "Formação X" possui uma relação de atravessar com o poço POCO_CD_POCO_024505 e possui a propriedade de ser constituída por sandstone. Não foram fornecidas informações adicionais no grafo.'}A formação que atravessa o poço com identificador POCO_CD_POCO_024505 e que é constituída por sandstone é a formação "Formação X".

> Finished chain.
{'input': 'Qual é o id da formação que atravessa o poço com identificador POCO_CD

## **Pregunta No. 8**

In [158]:
response = agent_executor.invoke({"input": "Quais unidades litoestratigraficas ou formações atravessam o poço _id:POCO_CD_POCO_024505  e são constituídas pelo id oil?"})
print(response)




> Entering new AgentExecutor chain...

Invoking: `Graph` with `Quais unidades litoestratigraficas ou formações atravessam o poço _id:POCO_CD_POCO_024505 e são constituídas pelo id oil?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (poco:well {_id: "POCO_CD_POCO_024505"})-[:crosses]->(unit:lithostratigraphic_unit)-[:constituted_by]->(:owl_NamedIndividual {_id: "oil"})
RETURN unit
Full Context:
[]

> Finished chain.
{'query': 'Quais unidades litoestratigraficas ou formações atravessam o poço _id:POCO_CD_POCO_024505 e são constituídas pelo id oil?', 'result': 'Não foi possível encontrar a informação requerida no grafo dado.'}Desculpe, não foi possível encontrar a informação sobre as unidades litoestratigráficas ou formações que atravessam o poço POCO_CD_POCO_024505 e são constituídas pelo id oil. Se precisar de mais alguma informação ou assistência, por favor me avise.

> Finished chain.
{'input': 'Quais unidades litoestratigraficas ou formações atravessam o po

# ✏️📚 **Conclusão**

Grafos de conhecimento são estruturas de dados que combinam dados estruturados e não estruturados de maneira interconectada, tornando-se uma excelente opção para alimentar aplicações RAG (Retrieval-Augmented Generation). Eles melhoram a precisão, contextualização e relevância das respostas geradas, aproveitando ao máximo as capacidades de recuperação e geração de informações.

### **Vantagens dos Grafos de Conhecimento:**

### 1. Relações Interconectadas:

- Grafos de conhecimento modelam relações complexas entre entidades (como pessoas, peliculas, no caso do grafo de Movies; e baçias, campos e poços, no caso do PetroKGraph), o que permite uma compreensão mais profunda e contextualizada dos dados.
- São ideais para representar conhecimento que envolve múltiplas relações e hierarquias.

### 2. Integração de Dados Estruturados e Não Estruturados:

- Permitem a integração de dados estruturados (como tabelas e bancos de dados relacionais) e dados não estruturados (como texto, imagens, e vídeos) em uma única estrutura coerente.
- Dados não estruturados podem ser anotados e vinculados a entidades no grafo, facilitando a recuperação contextual.

### 3. Consultas Flexíveis:

- Usando linguagens de consulta como Cypher, é possível executar consultas complexas que exploram as relações e conexões entre diferentes entidades.
- Isso possibilita a recuperação precisa de informações relevantes, mesmo em cenários com dados altamente interconectados.

### 4. Atualização Dinâmica e Enriquecimento de Dados:

- Grafos de conhecimento podem ser continuamente atualizados e enriquecidos com novos dados, mantendo a base de conhecimento atualizada.
- Facilitam a incorporação de novas fontes de dados, melhorando a abrangência e a qualidade das respostas.